In [16]:
%cd pro404

/media/notebooks/pro404


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [17]:
!hdfs dfs -mkdir /pro404

mkdir: `/pro404': File exists


In [18]:
!hdfs dfs -put ./countries_gdp_hist.csv /pro404
!hdfs dfs -put ./organizations_gdp_hist.csv /pro404


put: `/pro404/countries_gdp_hist.csv': File exists
put: `/pro404/organizations_gdp_hist.csv': File exists


## Ejercicio 1:  Limpieza y Transformación

In [19]:
%%writefile mapper1.py
#!/usr/bin/env python3

import sys 
first = True


for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(";")

    country_name = line[4]
    year = int(line[6])
    total_gdp = float(line[7])

    if year < 2000 or total_gdp <= 0:
        continue
    
    print(f"{country_name}\t{year}\t{total_gdp}")

Overwriting mapper1.py


In [20]:
%%writefile reducer1.py
#!/usr/bin/env python3

import sys

for line in sys.stdin:
    print(line)

Overwriting reducer1.py


In [21]:
!cat countries_gdp_hist.csv | python3 mapper1.py | sort | python3 reducer1.py

AFGHANISTAN	2000	3521418059.92345

AFGHANISTAN	2001	2813571753.87253

AFGHANISTAN	2002	3825701438.99963

AFGHANISTAN	2003	4520946818.54581

AFGHANISTAN	2004	5224896718.67782

AFGHANISTAN	2005	6203256538.70967

AFGHANISTAN	2006	6971758282.29351

AFGHANISTAN	2007	9747886187.39393

AFGHANISTAN	2008	10109297047.5432

AFGHANISTAN	2009	12416152732.0567

AFGHANISTAN	2010	15856668555.8336

AFGHANISTAN	2011	17805098206.3141

AFGHANISTAN	2012	19907329777.5872

AFGHANISTAN	2013	20146416757.5987

AFGHANISTAN	2014	20497128555.6972

AFGHANISTAN	2015	19134221644.7325

AFGHANISTAN	2016	18116572395.0772

AFGHANISTAN	2017	18753456497.8159

AFGHANISTAN	2018	18053222687.4126

AFGHANISTAN	2019	18799444490.1128

AFGHANISTAN	2020	19955929052.1496

AFGHANISTAN	2021	14259995441.0759

AFGHANISTAN	2022	14497243872.1337

AFGHANISTAN	2023	17233051620.1117

ALBANIA	2000	3480355258.04122

ALBANIA	2001	3922100793.5403

ALBANIA	2002	4348068242.19512

ALBANIA	2003	5611496257.14231

ALBANIA	2004	7184685781.51876

ALBANI

In [22]:
!hdfs dfs -rm -r /salida/pro404

Deleted /salida/pro404


In [23]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper1.py -file reducer1.py -mapper mapper1.py -reducer reducer1.py -input /pro404/countries_gdp_hist.csv -output /salida/pro404

2026-01-13 16:33:36,482 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper1.py, reducer1.py, /tmp/hadoop-unjar6459586874292243483/] [] /tmp/streamjob1131236790684078311.jar tmpDir=null
2026-01-13 16:33:37,079 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:33:37,183 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:33:37,363 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1768321127201_0003
2026-01-13 16:33:37,727 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-13 16:33:37,795 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-13 16:33:37,897 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1768321127201_0003
2026-01-13 16:33:37,897 INFO mapreduce.JobSubmitter: Executing 

In [24]:
!hdfs dfs -cat /salida/pro404/part-00000

AFGHANISTAN	2023	17233051620.1117
	
AFGHANISTAN	2000	3521418059.92345
	
AFGHANISTAN	2001	2813571753.87253
	
AFGHANISTAN	2002	3825701438.99963
	
AFGHANISTAN	2003	4520946818.54581
	
AFGHANISTAN	2004	5224896718.67782
	
AFGHANISTAN	2005	6203256538.70967
	
AFGHANISTAN	2006	6971758282.29351
	
AFGHANISTAN	2007	9747886187.39393
	
AFGHANISTAN	2008	10109297047.5432
	
AFGHANISTAN	2009	12416152732.0567
	
AFGHANISTAN	2010	15856668555.8336
	
AFGHANISTAN	2011	17805098206.3141
	
AFGHANISTAN	2012	19907329777.5872
	
AFGHANISTAN	2013	20146416757.5987
	
AFGHANISTAN	2014	20497128555.6972
	
AFGHANISTAN	2015	19134221644.7325
	
AFGHANISTAN	2016	18116572395.0772
	
AFGHANISTAN	2017	18753456497.8159
	
AFGHANISTAN	2018	18053222687.4126
	
AFGHANISTAN	2019	18799444490.1128
	
AFGHANISTAN	2020	19955929052.1496
	
AFGHANISTAN	2021	14259995441.0759
	
AFGHANISTAN	2022	14497243872.1337
	
ALBANIA	2000	3480355258.04122
	
ALBANIA	2023	23547179830.4413
	
ALBANIA	2022	19017242585.7803
	
ALBANIA	2021	18032010563.7558
	
ALBANIA	

In [25]:
%%writefile mapper2.py
#!/usr/bin/env python3

import sys 
first = True


for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(";")
    
    region_name = line[1]
    total_gdp = float(line[7])

    
    print(f"{region_name}\t{total_gdp}")

Overwriting mapper2.py


In [26]:
%%writefile reducer2.py
#!/usr/bin/env python3

import sys

cont = 0
total = 0
region_actual = None

for line in sys.stdin:
    line = line.strip().split()

    region = line[0]
    gdp = float(line[1])

    if region_actual == None:
        region_actual = region

    if region != region_actual:
        prom = total / cont 
        print(f"{region_actual}  {prom}")
        
        region_actual = region
        total = gdp    
        cont = 1

    if region_actual == region:
        total = total + gdp
        cont = cont + 1 

if region_actual is not None:
    prom = total / cont
    print(f"{region_actual}\t{prom}")

Overwriting reducer2.py


In [27]:
!cat countries_gdp_hist.csv | python3 mapper2.py | sort | python3 reducer2.py


AFRICA  17415301365.064095
AMERICAS  250035838103.24847
ASIA  205848965958.85745
EUROPE  213106710082.61438
OCEANIA	32506279297.297432


In [28]:
%%writefile mapper3.py
#!/usr/bin/env python3

import sys 
first = True


for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(";")

    country_name = line[4]
    year = int(line[6])
    variation = float(line[9])

    yearVariation = f"{year}"+","+f"{variation}"

    print(f"{country_name}\t{yearVariation}")

Overwriting mapper3.py


In [29]:
%%writefile reducer3.py
#!/usr/bin/env python3

import sys 

pais_actual = None
maxVariacion = 0
año = 0

for line in sys.stdin:

    line = line.strip().split("\t")

    pais = line[0]
    yearVariation = line[1]

    yearVariation = yearVariation.strip().split(",")

    year = int(yearVariation[0])
    variation = float(yearVariation[1])

    if pais_actual == None:
        pais_actual = pais

    if pais_actual == pais: 
        if variation > maxVariacion:
            maxVariacion = variation
            año = year

    if pais_actual != pais:
        print(f"{pais_actual} \t {año} ({maxVariacion})")

        pais_actual = pais
        max_variacion = variation
        año_record = year

Overwriting reducer3.py


In [30]:
!cat countries_gdp_hist.csv | python3 mapper3.py | sort | python3 reducer3.py

AFGHANISTAN 	 2002 (28.6000011706788)
ALBANIA 	 2002 (28.6000011706788)
ALGERIA 	 1963 (34.3137287803281)
AMERICAN SAMOA 	 1963 (34.3137287803281)
ANDORRA 	 1963 (34.3137287803281)
ANGOLA 	 1963 (34.3137287803281)
ANTIGUA AND BARBUDA 	 1963 (34.3137287803281)
ARGENTINA 	 1963 (34.3137287803281)
ARMENIA 	 1963 (34.3137287803281)
ARUBA 	 1963 (34.3137287803281)
AUSTRALIA 	 1963 (34.3137287803281)
AUSTRIA 	 1963 (34.3137287803281)
AZERBAIJAN 	 2006 (34.5)
BAHAMAS 	 2006 (34.5)
BAHRAIN 	 2006 (34.5)
BANGLADESH 	 2006 (34.5)
BARBADOS 	 2006 (34.5)
BELARUS 	 2006 (34.5)
BELGIUM 	 2006 (34.5)
BELIZE 	 2006 (34.5)
BENIN 	 2006 (34.5)
BERMUDA 	 2006 (34.5)
BHUTAN 	 2006 (34.5)
BOLIVIA (PLURINATIONAL STATE OF) 	 2006 (34.5)
BOSNIA AND HERZEGOVINA 	 1996 (54.1993001671129)
BOTSWANA 	 1996 (54.1993001671129)
BRAZIL 	 1996 (54.1993001671129)
BRUNEI DARUSSALAM 	 1996 (54.1993001671129)
BULGARIA 	 1996 (54.1993001671129)
BURKINA FASO 	 1996 (54.1993001671129)
BURUNDI 	 1996 (54.1993001671129)
CABO VE

In [31]:
!hdfs dfs -rm -r /salida/pro404

Deleted /salida/pro404


In [32]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper3.py -file reducer3.py -mapper mapper3.py -reducer reducer3.py -input /pro404/countries_gdp_hist.csv -output /salida/pro404

2026-01-13 16:33:57,943 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper3.py, reducer3.py, /tmp/hadoop-unjar6263963667802401878/] [] /tmp/streamjob3756041134533067482.jar tmpDir=null
2026-01-13 16:33:58,581 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:33:58,720 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:33:58,894 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1768321127201_0004
2026-01-13 16:33:59,234 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-13 16:33:59,292 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-13 16:33:59,376 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1768321127201_0004
2026-01-13 16:33:59,377 INFO mapreduce.JobSubmitter: Executing 

In [33]:
!hdfs dfs -cat /salida/pro404/part-00000

AFGHANISTAN 	 2002 (28.6000011706788)
ALBANIA 	 2002 (28.6000011706788)
ALGERIA 	 2002 (28.6000011706788)
AMERICAN SAMOA 	 2002 (28.6000011706788)
ANDORRA 	 2002 (28.6000011706788)
ANGOLA 	 2002 (28.6000011706788)
ANTIGUA AND BARBUDA 	 2002 (28.6000011706788)
ARGENTINA 	 2002 (28.6000011706788)
ARMENIA 	 2002 (28.6000011706788)
ARUBA 	 2002 (28.6000011706788)
AUSTRALIA 	 2002 (28.6000011706788)
AUSTRIA 	 2002 (28.6000011706788)
AZERBAIJAN 	 2006 (34.5)
BAHAMAS 	 2006 (34.5)
BAHRAIN 	 2006 (34.5)
BANGLADESH 	 2006 (34.5)
BARBADOS 	 2006 (34.5)
BELARUS 	 2006 (34.5)
BELGIUM 	 2006 (34.5)
BELIZE 	 2006 (34.5)
BENIN 	 2006 (34.5)
BERMUDA 	 2006 (34.5)
BHUTAN 	 2006 (34.5)
BOLIVIA (PLURINATIONAL STATE OF) 	 2006 (34.5)
BOSNIA AND HERZEGOVINA 	 1996 (54.1993001671129)
BOTSWANA 	 1996 (54.1993001671129)
BRAZIL 	 1996 (54.1993001671129)
BRUNEI DARUSSALAM 	 1996 (54.1993001671129)
BULGARIA 	 1996 (54.1993001671129)
BURKINA FASO 	 1996 (54.1993001671129)
BURUNDI 	 1996 (54.1993001671129)
CABO VE

In [34]:
%%writefile mapper4.py
#!/usr/bin/env python3
 
import sys
 
first = True
codes = False
 
for line in sys.stdin:
    if first:
        first = False
        continue
 
    fich = line.strip().split(",")
    if len(fich) > 2:
        codes = True
    else:
        codes = False
   
    if codes:
        line = line.strip().split(",")
 
        code = line[2]
        name = line[0]
        print(f"{code}\tA_{name}")
    else:
        line = line.strip().split(";")
 
        code = line[0]
        pib = line[8]
        print(f"{code}\tB_{pib}")

Overwriting mapper4.py


In [35]:
%%writefile reducer4.py
#!/usr/bin/env python3
 
import sys
 
codes = {}
 
for line in sys.stdin:
    line = line.strip().split()
    cod = line[0]
    rest = line[1]
 
    rest = rest.strip().split("_")
    if rest[0] == "A":
        codes[cod] = rest[1]
    elif rest[0] == "B":
        if cod in codes:
            print(f"{codes[cod]}\t{rest[1]}")

Overwriting reducer4.py


In [36]:
!hdfs dfs -put ./all.csv /pro404

put: `/pro404/all.csv': File exists


In [37]:
!hdfs dfs -rm -r /salida/pro404

Deleted /salida/pro404


In [38]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper4.py -file reducer4.py -mapper mapper4.py -reducer reducer4.py -input /pro404/countries_gdp_hist.csv -input /pro404/all.csv -output /salida/pro404

2026-01-13 16:34:19,528 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper4.py, reducer4.py, /tmp/hadoop-unjar6013782882232105604/] [] /tmp/streamjob6379025987771455466.jar tmpDir=null
2026-01-13 16:34:20,106 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:34:20,228 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:34:20,390 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1768321127201_0005
2026-01-13 16:34:20,715 INFO mapred.FileInputFormat: Total input files to process : 2
2026-01-13 16:34:20,802 INFO mapreduce.JobSubmitter: number of splits:3
2026-01-13 16:34:20,891 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1768321127201_0005
2026-01-13 16:34:20,891 INFO mapreduce.JobSubmitter: Executing 

In [39]:
!hdfs dfs -cat /salida/pro404/part-00000

Aruba	1873.45251396648
Aruba	1961.84357541899
Aruba	2044.11173184358
Aruba	2254.83072625698
Aruba	2360.01731843575
Aruba	2469.78268156425
Aruba	2677.64134078212
Aruba	2843.02458100559
Aruba	2553.79329608939
Aruba	2453.5972067039097
Aruba	2637.85921787709
Aruba	2615.20837988827
Aruba	0.0
Aruba	2727.84972067039
Aruba	2790.84972067039
Aruba	2962.90726256983
Aruba	2983.6351955307296
Aruba	3092.4290502793297
Aruba	3276.1843575419
Aruba	3395.79888268156
Aruba	2481.8571228400997
Aruba	2929.44657825104
Aruba	3279.34354360416
Aruba	3648.57313615155
Aruba	1896.45698324022
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	0.0
Aruba	405.586592178771
Aruba	487.70949720670404
Aruba	596.648044692737
Aruba	695.530726256983
Aruba	764.804469273743
Aruba	872.067039106145
Aruba	958.659217877095
Aruba	1083.2402234

In [40]:
%%writefile mapper5.py
#!/usr/bin/env python3

import sys 
first = True


for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(";")

    million_gdp = float(line[8])

    if million_gdp < 10000:
        print(f"Economía Pequeña\t1")
    elif million_gdp < 100000:
        print(f"Economía Mediana\t1")
    else:
        print(f"Economía Grande\t1")
    

Overwriting mapper5.py


In [41]:
%%writefile reducer5.py
#!/usr/bin/env python3

import sys 

cat_act = None
total = 0

for line in sys.stdin:
    line = line.strip().split("\t")

    cat = line[0]
    val = int(line[1])

    if cat_act == None:
        cat_act = cat
    
    if cat_act == cat:
        total = total + val

    if cat_act != cat:
        print(f"{cat_act} {total}")
        cat_act = cat 
        total = val
    
print(f"{cat_act} {total}")    

Overwriting reducer5.py


In [42]:
!cat countries_gdp_hist.csv | python3 mapper5.py | sort | python3 reducer5.py

Economía Grande 2042
Economía Mediana 3158
Economía Pequeña 8560


In [43]:
!hdfs dfs -rm -r /salida/pro404

Deleted /salida/pro404


In [44]:
!hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar -file mapper5.py -file reducer5.py -mapper mapper5.py -reducer reducer5.py -input /pro404/countries_gdp_hist.csv  -output /salida/pro404

2026-01-13 16:34:39,887 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper5.py, reducer5.py, /tmp/hadoop-unjar4257934932790643450/] [] /tmp/streamjob2465518640857866631.jar tmpDir=null
2026-01-13 16:34:40,666 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:34:40,881 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.20.0.5:8032
2026-01-13 16:34:41,193 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1768321127201_0006
2026-01-13 16:34:41,677 INFO mapred.FileInputFormat: Total input files to process : 1
2026-01-13 16:34:41,755 INFO mapreduce.JobSubmitter: number of splits:2
2026-01-13 16:34:41,865 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1768321127201_0006
2026-01-13 16:34:41,865 INFO mapreduce.JobSubmitter: Executing 

In [45]:
!hdfs dfs -cat /salida/pro404/part-00000

Economía Grande 2042	
Economía Mediana 3157	
Economía Pequeña 8560	


In [46]:
%%writefile mapper6.py
#!/usr/bin/env python3

import sys

first = True


for line in sys.stdin:
    if first == True:
        first = False
        continue
    
    line = line.strip().split(";")

    country_name = line[4]
    income_group = line[5]

    print(f"{income_group}\t{country_name}")



Overwriting mapper6.py


In [47]:
%%writefile reducer6.py
#!/usr/bin/env python3

import sys

grupo_actual = None
paises_set = set()

for line in sys.stdin:
    line = line.strip().split("\t")
        
    grupo = line[0]
    pais =  line[1]
    
    if grupo_actual == grupo:
        paises_set.add(pais)
    
    else:   
        if grupo_actual is not None:
            lista_paises = ", ".join(sorted(paises_set))
            print(f"{grupo_actual}\t{lista_paises}")
        
        grupo_actual = grupo
        paises_set = {pais}

if grupo_actual is not None:
    lista_paises = ", ".join(sorted(paises_set))
    print(f"{grupo_actual}\t{lista_paises}")


Overwriting reducer6.py


In [48]:
!cat countries_gdp_hist.csv | python3 mapper6.py | sort | python3 reducer6.py

INGRESO ALTO	AMERICAN SAMOA, ANDORRA, ANTIGUA AND BARBUDA, ARUBA, AUSTRALIA, AUSTRIA, BAHAMAS, BAHRAIN, BARBADOS, BELGIUM, BERMUDA, BRUNEI DARUSSALAM, BULGARIA, CANADA, CAYMAN ISLANDS, CHILE, CROATIA, CURAÇAO, CYPRUS, CZECHIA, DENMARK, ESTONIA, FAROE ISLANDS, FINLAND, FRANCE, FRENCH POLYNESIA, GERMANY, GIBRALTAR, GREECE, GREENLAND, GUAM, GUYANA, HONG KONG, HUNGARY, ICELAND, IRELAND, ISLE OF MAN, ISRAEL, ITALY, JAPAN, KOREA, REPUBLIC OF, KUWAIT, LATVIA, LIECHTENSTEIN, LITHUANIA, LUXEMBOURG, MACAO, MALTA, MONACO, NAURU, NETHERLANDS, NEW CALEDONIA, NEW ZEALAND, NORTHERN MARIANA ISLANDS, NORWAY, OMAN, PALAU, PANAMA, POLAND, PORTUGAL, PUERTO RICO, QATAR, ROMANIA, RUSSIAN FEDERATION, SAINT KITTS AND NEVIS, SAINT MARTIN (FRENCH PART), SAN MARINO, SAUDI ARABIA, SEYCHELLES, SINGAPORE, SINT MAARTEN (DUTCH PART), SLOVAKIA, SLOVENIA, SPAIN, SWEDEN, SWITZERLAND, TRINIDAD AND TOBAGO, TURKS AND CAICOS ISLANDS, UNITED ARAB EMIRATES, UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND, UNITED STATES O